# Parallel Programming with MPI and mpi4py

**Computational Physics 2025**  
**Lecture 7: Message Passing Interface (MPI)**  
**Date: November 21, 2025**

---

## Lecture Outline (60 minutes)

1. **Introduction to MPI** (10 min)
   - What is MPI and why use it?
   - MPI vs. other parallelization methods
   - Installing and setting up mpi4py

2. **Basic MPI Concepts** (15 min)
   - Communicators, ranks, and size
   - Point-to-point communication
   - Collective communication

3. **Practical Examples** (20 min)
   - Hello World in MPI
   - Data distribution and gathering
   - Parallel numerical integration
   - Parallel matrix operations

4. **Advanced Topics** (10 min)
   - Non-blocking communication
   - Custom datatypes
   - Performance considerations

5. **Physics Applications** (5 min)
   - Monte Carlo simulations
   - N-body problems
   - Partial differential equations

---

## 1. Introduction to MPI

### What is MPI?

**Message Passing Interface (MPI)** is a standardized and portable message-passing system designed to function on parallel computing architectures.

**Key Features:**
- Industry standard for distributed-memory parallel computing
- Designed for high-performance computing (HPC)
- Works on clusters, supercomputers, and multi-core machines
- Language-independent (C, C++, Fortran, Python via mpi4py)

### Why Use MPI?

1. **Scalability**: Can scale from laptops to supercomputers with thousands of nodes
2. **Performance**: Minimal overhead, designed for efficiency
3. **Flexibility**: Fine-grained control over communication patterns
4. **Portability**: Code runs on different architectures with minimal changes

### MPI vs. Other Parallel Programming Models

| Feature | MPI | OpenMP | Threading |
|---------|-----|--------|-----------|
| Memory Model | Distributed | Shared | Shared |
| Scalability | Excellent | Limited | Limited |
| Programming Complexity | Higher | Lower | Medium |
| Best For | Multi-node clusters | Single multi-core machine | Single multi-core machine |

### Installing mpi4py

**Prerequisites:** You need an MPI implementation installed:
- **macOS**: `brew install openmpi`
- **Linux**: `sudo apt-get install openmpi-bin libopenmpi-dev` (Ubuntu/Debian)
- **Windows**: Use Microsoft MPI or WSL

**Install mpi4py:**
```bash
pip install mpi4py
```

In [3]:
# Check if mpi4py is installed
try:
    import mpi4py
    print(f"mpi4py version: {mpi4py.__version__}")
    print("MPI is ready to use!")
except ImportError:
    print("mpi4py is not installed. Please run: pip install mpi4py")

mpi4py version: 4.1.1
MPI is ready to use!


---

## 2. Basic MPI Concepts

### Fundamental MPI Terminology

1. **Communicator**: A group of processes that can communicate with each other
   - `MPI.COMM_WORLD`: Default communicator containing all processes

2. **Rank**: Unique identifier for each process (0, 1, 2, ..., N-1)

3. **Size**: Total number of processes in a communicator

### The MPI Execution Model

<pre>
┌─────────────────────────────────────────────┐
│  Same Program, Multiple Data (SPMD)         │
├─────────────────────────────────────────────┤
│  Rank 0  │  Rank 1  │  Rank 2  │  Rank 3    │
│  ┌────┐  │  ┌────┐  │  ┌────┐  │  ┌────┐    │
│  │Code│  │  │Code│  │  │Code│  │  │Code│    │
│  └────┘  │  └────┘  │  └────┘  │  └────┘    │
│  Data A  │  Data B  │  Data C  │  Data D    │
└─────────────────────────────────────────────┘
</pre>
**Key Principle**: All processes execute the same code, but can perform different operations based on their rank.

### Communication Patterns

1. **Point-to-Point Communication**
   - `send()` / `recv()`: Send and receive data between specific processes
   - `Send()` / `Recv()`: Capitalized versions for numpy arrays (faster)

2. **Collective Communication**
   - `bcast()`: Broadcast data from one process to all
   - `scatter()`: Distribute different data to each process
   - `gather()`: Collect data from all processes to one
   - `reduce()`: Combine data from all processes using an operation (sum, max, etc.)

### Basic MPI Program Structure

Every MPI program in Python follows this pattern:

```python
from mpi4py import MPI

# Get the communicator
comm = MPI.COMM_WORLD

# Get rank and size
rank = comm.Get_rank()
size = comm.Get_size()

# Your parallel code here
# Different ranks can do different things
if rank == 0:
    # Master process
    pass
else:
    # Worker processes
    pass
```

**Important Note**: MPI programs **cannot** be run directly in Jupyter notebooks. They must be executed using the `mpiexec` or `mpirun` command from the terminal.

Let's create our first MPI program:

In [4]:
# Create a simple MPI hello world program
mpi_hello_code = """from mpi4py import MPI
import socket

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
hostname = socket.gethostname()

# Each process prints its information
print(f"Hello from rank {rank} of {size} processes on host {hostname}")

# Synchronize all processes
comm.Barrier()

# Only rank 0 prints the summary
if rank == 0:
    print(f"\\n{'='*50}")
    print(f"MPI program completed with {size} processes")
    print(f"{'='*50}")
"""

# Write to file
with open('mpi_hello_example.py', 'w') as f:
    f.write(mpi_hello_code)

print("Created: mpi_hello_example.py")
print("\nTo run this MPI program, use the terminal command:")
print("  mpiexec -n 4 python mpi_hello_example.py")
print("\nwhere -n 4 specifies 4 processes")

Created: mpi_hello_example.py

To run this MPI program, use the terminal command:
  mpiexec -n 4 python mpi_hello_example.py

where -n 4 specifies 4 processes


---

## 3. Point-to-Point Communication

### Send and Receive

Point-to-point communication involves sending data from one process to another specific process.

**Two flavors in mpi4py:**

1. **Lowercase methods** (`send`, `recv`): For general Python objects
   - More flexible but slower
   - Uses pickle serialization
   
2. **Uppercase methods** (`Send`, `Recv`): For NumPy arrays
   - Much faster (no serialization)
   - Requires contiguous memory buffers

### Example: Sending Python Objects

In [5]:
# Example: Point-to-point communication
send_recv_code = """from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if size < 2:
    print("This example requires at least 2 processes")
    exit()

# Example 1: Send/receive Python objects (lowercase)
if rank == 0:
    data = {'message': 'Hello from rank 0', 'value': 42, 'array': [1, 2, 3]}
    comm.send(data, dest=1, tag=11)
    print(f"Rank 0 sent: {data}")
elif rank == 1:
    data = comm.recv(source=0, tag=11)
    print(f"Rank 1 received: {data}")

comm.Barrier()

# Example 2: Send/receive NumPy arrays (uppercase - FASTER!)
if rank == 0:
    array = np.array([1.0, 2.0, 3.0, 4.0, 5.0], dtype=np.float64)
    comm.Send(array, dest=1, tag=22)
    print(f"\\nRank 0 sent NumPy array: {array}")
elif rank == 1:
    array = np.empty(5, dtype=np.float64)
    comm.Recv(array, source=0, tag=22)
    print(f"Rank 1 received NumPy array: {array}")
"""

with open('mpi_send_recv.py', 'w') as f:
    f.write(send_recv_code)

print("Created: mpi_send_recv.py")
print("\nRun with: mpiexec -n 2 python mpi_send_recv.py")

Created: mpi_send_recv.py

Run with: mpiexec -n 2 python mpi_send_recv.py


---

## 4. Collective Communication

Collective operations involve **all** processes in a communicator. These are highly optimized and are the backbone of efficient parallel algorithms.

### 4.1 Broadcast (`bcast`)

Sends data from one process (root) to all other processes.

<pre>
Before:           After:
Rank 0: [A]       Rank 0: [A]
Rank 1: [ ]   →   Rank 1: [A]
Rank 2: [ ]       Rank 2: [A]
Rank 3: [ ]       Rank 3: [A]
</pre>

### 4.2 Scatter (`scatter`)

Distributes different portions of data to each process.

<pre>
Before:                After:
Rank 0: [A,B,C,D]      Rank 0: [A]
Rank 1: [ ]        →   Rank 1: [B]
Rank 2: [ ]            Rank 2: [C]
Rank 3: [ ]            Rank 3: [D]
</pre>

### 4.3 Gather (`gather`)

Collects data from all processes to one process.

<pre>
Before:           After:
Rank 0: [A]       Rank 0: [A,B,C,D]
Rank 1: [B]   →   Rank 1: [B]
Rank 2: [C]       Rank 2: [C]
Rank 3: [D]       Rank 3: [D]
</pre>

### 4.4 Reduce (`reduce`)

Combines data from all processes using an operation (SUM, MAX, MIN, etc.).

<pre>
Before:           After (SUM):
Rank 0: [1]       Rank 0: [10]
Rank 1: [2]   →   Rank 1: [2]
Rank 2: [3]       Rank 2: [3]
Rank 3: [4]       Rank 3: [4]
</pre>

In [6]:
# Example: Collective communication operations
collective_code = """from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

print(f"\\n{'='*60}")
print(f"Process {rank} starting collective communication examples")
print(f"{'='*60}")

# Example 1: Broadcast
if rank == 0:
    data = {'param': 'simulation_config', 'timesteps': 1000, 'dt': 0.01}
    print(f"\\nRank 0 broadcasting: {data}")
else:
    data = None

data = comm.bcast(data, root=0)
print(f"Rank {rank} received broadcast: {data}")

comm.Barrier()

# Example 2: Scatter
if rank == 0:
    # Split work among processes
    work = np.arange(size * 4).reshape(size, 4)
    print(f"\\nRank 0 scattering work:\\n{work}")
else:
    work = None

local_work = comm.scatter(work, root=0)
print(f"Rank {rank} received: {local_work}")

comm.Barrier()

# Example 3: Gather
# Each process computes something
local_result = (rank + 1) ** 2
print(f"\\nRank {rank} computed: {local_result}")

all_results = comm.gather(local_result, root=0)
if rank == 0:
    print(f"Rank 0 gathered all results: {all_results}")

comm.Barrier()

# Example 4: Reduce (sum)
local_value = np.array([rank + 1], dtype=np.float64)
total = np.zeros(1, dtype=np.float64)

comm.Reduce(local_value, total, op=MPI.SUM, root=0)

if rank == 0:
    print(f"\\nSum of all ranks (1+2+...+{size}): {total[0]}")
    print(f"Expected: {size * (size + 1) // 2}")

# Example 5: Allreduce (result available on all processes)
local_array = np.ones(3) * (rank + 1)
global_sum = np.zeros(3)

comm.Allreduce(local_array, global_sum, op=MPI.SUM)
print(f"Rank {rank} sees global sum: {global_sum}")
"""

with open('mpi_collective.py', 'w') as f:
    f.write(collective_code)

print("Created: mpi_collective.py")
print("\nRun with: mpiexec -n 4 python mpi_collective.py")

Created: mpi_collective.py

Run with: mpiexec -n 4 python mpi_collective.py


---

## 5. Practical Application: Parallel Numerical Integration

Let's compute π using the Monte Carlo method and parallel trapezoidal integration.

### Monte Carlo π Calculation

The idea: 
- Generate random points in a unit square
- Count how many fall inside a quarter circle
- π ≈ 4 × (points inside circle) / (total points)

Each process generates its own random points, then we sum the results.

In [7]:
# Example: Parallel Monte Carlo calculation of π
pi_monte_carlo = """from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Total number of samples
N_total = 100_000_000

# Each process handles a portion
N_local = N_total // size

# Start timing
start_time = time.time()

# Generate random points in unit square
np.random.seed(rank)  # Different seed for each process
x = np.random.random(N_local)
y = np.random.random(N_local)

# Count points inside quarter circle
inside = np.sum(x**2 + y**2 <= 1.0)

# Sum results from all processes
total_inside = comm.reduce(inside, op=MPI.SUM, root=0)

end_time = time.time()

if rank == 0:
    pi_estimate = 4.0 * total_inside / N_total
    print(f"{'='*60}")
    print(f"Parallel Monte Carlo Estimation of π")
    print(f"{'='*60}")
    print(f"Number of processes: {size}")
    print(f"Total samples: {N_total:,}")
    print(f"Samples per process: {N_local:,}")
    print(f"\\nEstimated π: {pi_estimate:.10f}")
    print(f"Actual π:    {np.pi:.10f}")
    print(f"Error:       {abs(pi_estimate - np.pi):.10f}")
    print(f"\\nTime taken: {end_time - start_time:.4f} seconds")
    print(f"Samples/sec: {N_total / (end_time - start_time):,.0f}")
"""

with open('mpi_pi_monte_carlo.py', 'w') as f:
    f.write(pi_monte_carlo)

print("Created: mpi_pi_monte_carlo.py")
print("\nRun with different numbers of processes to see speedup:")
print("  mpiexec -n 1 python mpi_pi_monte_carlo.py")
print("  mpiexec -n 2 python mpi_pi_monte_carlo.py")
print("  mpiexec -n 4 python mpi_pi_monte_carlo.py")
print("  mpiexec -n 8 python mpi_pi_monte_carlo.py")

Created: mpi_pi_monte_carlo.py

Run with different numbers of processes to see speedup:
  mpiexec -n 1 python mpi_pi_monte_carlo.py
  mpiexec -n 2 python mpi_pi_monte_carlo.py
  mpiexec -n 4 python mpi_pi_monte_carlo.py
  mpiexec -n 8 python mpi_pi_monte_carlo.py


### Parallel Trapezoidal Integration

Compute the integral ∫₀¹ √(1-x²) dx = π/4 using the trapezoidal rule in parallel.

**Strategy:**
1. Divide the integration domain among processes
2. Each process computes its local integral
3. Sum the results using `reduce`

In [8]:
# Example: Parallel trapezoidal integration
trapezoid_code = """from mpi4py import MPI
import numpy as np
import time

def f(x):
    \"\"\"Function to integrate: sqrt(1-x^2)\"\"\"
    return np.sqrt(1 - x**2)

def trapezoidal_rule(a, b, n, func):
    \"\"\"Compute integral using trapezoidal rule\"\"\"
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = func(x)
    integral = h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])
    return integral

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Integration parameters
a = 0.0  # Lower bound
b = 1.0  # Upper bound
n_total = 10_000_000  # Total number of trapezoids

# Divide work among processes
n_local = n_total // size

# Each process handles a portion of the domain
local_a = a + rank * (b - a) / size
local_b = a + (rank + 1) * (b - a) / size

# Start timing
start_time = time.time()

# Compute local integral
local_integral = trapezoidal_rule(local_a, local_b, n_local, f)

# Sum all local integrals
total_integral = comm.reduce(local_integral, op=MPI.SUM, root=0)

end_time = time.time()

if rank == 0:
    # The integral equals π/4, so multiply by 4
    pi_estimate = 4.0 * total_integral
    print(f"{'='*60}")
    print(f"Parallel Trapezoidal Integration")
    print(f"{'='*60}")
    print(f"Number of processes: {size}")
    print(f"Total trapezoids: {n_total:,}")
    print(f"Trapezoids per process: {n_local:,}")
    print(f"\\nIntegral value: {total_integral:.10f}")
    print(f"Estimated π: {pi_estimate:.10f}")
    print(f"Actual π:    {np.pi:.10f}")
    print(f"Error:       {abs(pi_estimate - np.pi):.10f}")
    print(f"\\nTime taken: {end_time - start_time:.4f} seconds")
"""

with open('mpi_trapezoid.py', 'w') as f:
    f.write(trapezoid_code)

print("Created: mpi_trapezoid.py")
print("\nRun with: mpiexec -n 4 python mpi_trapezoid.py")

Created: mpi_trapezoid.py

Run with: mpiexec -n 4 python mpi_trapezoid.py


---

## 6. Parallel Matrix-Vector Multiplication

A common operation in scientific computing: **y = A × x**

**Parallel strategy:**
1. Distribute rows of matrix A among processes
2. Broadcast vector x to all processes
3. Each process computes its portion of the result
4. Gather results back to root process

In [9]:
# Example: Parallel matrix-vector multiplication
matvec_code = """from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Matrix dimensions
N = 1000  # Size of square matrix (reduced for faster execution)

# Number of rows per process
rows_per_process = N // size

# Create matrix and vector on root
if rank == 0:
    A = np.random.randn(N, N)
    x = np.random.randn(N)
    y_serial = np.zeros(N)
    
    # Serial computation for comparison
    start_serial = time.time()
    y_serial = A @ x
    end_serial = time.time()
    serial_time = end_serial - start_serial
    
    print(f"{'='*60}")
    print(f"Parallel Matrix-Vector Multiplication")
    print(f"{'='*60}")
    print(f"Matrix size: {N} × {N}")
    print(f"Number of processes: {size}")
    print(f"Rows per process: {rows_per_process}")
    print(f"\\nSerial time: {serial_time:.4f} seconds")
else:
    A = None
    x = None
    y_serial = None
    serial_time = 0.0

# Start parallel timing
comm.Barrier()
start_parallel = time.time()

# Scatter rows of A to all processes
if rank == 0:
    # Split A into chunks for each process
    A_chunks = [A[i*rows_per_process:(i+1)*rows_per_process] for i in range(size)]
else:
    A_chunks = None

local_A = comm.scatter(A_chunks, root=0)

# Broadcast x to all processes
x_local = comm.bcast(x if rank == 0 else None, root=0)

# Each process computes its portion of y
local_y = local_A @ x_local

# Gather results
y_parallel = comm.gather(local_y, root=0)
if rank == 0:
    y_parallel = np.concatenate(y_parallel)

comm.Barrier()
end_parallel = time.time()

if rank == 0:
    parallel_time = end_parallel - start_parallel
    
    print(f"\\nParallel time: {parallel_time:.4f} seconds")
    print(f"Speedup: {serial_time / parallel_time:.2f}x")
    print(f"Efficiency: {100 * serial_time / (parallel_time * size):.1f}%")
    
    # Verify correctness
    error = np.linalg.norm(y_serial - y_parallel) / np.linalg.norm(y_serial)
    print(f"\\nRelative error: {error:.2e}")
    if error < 1e-10:
        print("✓ Results match!")
    else:
        print("✗ Results differ!")
"""

with open('mpi_matvec.py', 'w') as f:
    f.write(matvec_code)

print("Created: mpi_matvec.py")
print("\nRun with: mpiexec -n 4 python mpi_matvec.py")
print("\nNote: This version uses lowercase scatter/bcast/gather for automatic")
print("      data handling. Matrix size reduced to 1000 for faster execution.")

Created: mpi_matvec.py

Run with: mpiexec -n 4 python mpi_matvec.py


---

## 7. Advanced Topics

### 7.1 Non-Blocking Communication

**Blocking vs Non-Blocking:**

- **Blocking** (`send`, `recv`): Process waits until operation completes
- **Non-blocking** (`isend`, `irecv`): Process continues immediately, can do other work

**Benefits:**
- Overlap communication with computation
- Avoid deadlocks in complex communication patterns
- Better performance in many scenarios

```python
# Non-blocking example
request = comm.isend(data, dest=1, tag=11)
# Do other work here...
request.wait()  # Wait for send to complete
```

### 7.2 Common MPI Operations

| Operation | Description | Example Use Case |
|-----------|-------------|------------------|
| `MPI.SUM` | Sum reduction | Total energy, particle count |
| `MPI.MAX` | Maximum value | Maximum temperature, error |
| `MPI.MIN` | Minimum value | Convergence check |
| `MPI.PROD` | Product | Determinants |
| `MPI.LAND` | Logical AND | Convergence flags |
| `MPI.LOR` | Logical OR | Error detection |

### 7.3 Performance Tips

1. **Use uppercase methods for NumPy arrays** (`Send`, `Recv`) - much faster
2. **Minimize communication** - computation should dominate
3. **Use collective operations** instead of loops of point-to-point
4. **Balance the load** - all processes should have similar work
5. **Consider data locality** - minimize data movement
6. **Profile your code** - identify bottlenecks

### 7.4 Common Pitfalls

⚠️ **Deadlock**: Processes waiting for each other indefinitely
```python
# BAD: Both wait to receive before sending
data = comm.recv(source=other_rank)
comm.send(my_data, dest=other_rank)

# GOOD: One sends first, other receives first
if rank == 0:
    comm.send(my_data, dest=1)
    data = comm.recv(source=1)
else:
    data = comm.recv(source=0)
    comm.send(my_data, dest=0)
```

⚠️ **Load imbalance**: Some processes finish much earlier than others

⚠️ **Communication overhead**: Too much communication, not enough computation

In [10]:
# Example: Non-blocking communication
nonblocking_code = """from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if size < 2:
    print("This example requires at least 2 processes")
    exit()

N = 1_000_000
data_to_send = np.arange(N, dtype=np.float64) * (rank + 1)
data_to_recv = np.empty(N, dtype=np.float64)

# Non-blocking send/receive
if rank == 0:
    # Send to rank 1, receive from rank 1
    req_send = comm.Isend(data_to_send, dest=1, tag=0)
    req_recv = comm.Irecv(data_to_recv, source=1, tag=1)
    
    # Do some computation while communication happens
    start = time.time()
    local_result = np.sum(data_to_send ** 2)
    computation_time = time.time() - start
    
    # Wait for communication to complete
    req_send.wait()
    req_recv.wait()
    
    print(f"Rank 0: Communication overlapped with computation!")
    print(f"Computation took: {computation_time:.6f} seconds")
    print(f"Received data sum: {np.sum(data_to_recv):.2e}")
    
elif rank == 1:
    # Receive from rank 0, send to rank 0
    req_recv = comm.Irecv(data_to_recv, source=0, tag=0)
    req_send = comm.Isend(data_to_send, dest=0, tag=1)
    
    # Do some computation while communication happens
    local_result = np.sum(data_to_send ** 2)
    
    # Wait for communication to complete
    req_send.wait()
    req_recv.wait()
    
    print(f"Rank 1: Received data sum: {np.sum(data_to_recv):.2e}")
"""

with open('mpi_nonblocking.py', 'w') as f:
    f.write(nonblocking_code)

print("Created: mpi_nonblocking.py")
print("\nRun with: mpiexec -n 2 python mpi_nonblocking.py")

Created: mpi_nonblocking.py

Run with: mpiexec -n 2 python mpi_nonblocking.py


---

## 8. Physics Applications with MPI

### 8.1 Monte Carlo Simulations

**Perfect for MPI parallelization!**

Examples:
- **Ising Model**: Each process simulates independent configurations
- **Particle Transport**: Each process tracks different particles
- **Quantum Monte Carlo**: Distribute walkers among processes

**Key idea**: Independent random trajectories → embarrassingly parallel

### 8.2 N-Body Simulations

**Challenge**: All particles interact with each other

**Strategies:**
1. **Domain Decomposition**: Divide space into regions
2. **Particle Decomposition**: Distribute particles among processes
3. **Force Decomposition**: Distribute force calculations

**Communication pattern**: 
- Exchange boundary information between neighboring domains
- Use `Allgather` for small N, domain decomposition for large N

### 8.3 Partial Differential Equations (PDEs)

**Examples**: Heat equation, wave equation, Schrödinger equation

**Approach:**
1. Discretize domain into grid
2. Distribute grid points among processes
3. Each process updates its local points
4. Exchange boundary values with neighbors

**Pattern**: Nearest-neighbor communication (ghost cells)

### Example: 1D Heat Equation

The heat equation: ∂u/∂t = α ∂²u/∂x²

We'll solve this using finite differences with domain decomposition.

In [ ]:
# Example: Parallel 1D heat equation solver
heat_eq_code = """from mpi4py import MPI
import numpy as np
import matplotlib.pyplot as plt

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Physical parameters
L = 1.0          # Length of domain
alpha = 0.01     # Thermal diffusivity
T_final = 0.5    # Final time

# Numerical parameters
N_global = 1004  # Total number of grid points (must be divisible by number of processes)
dt = 0.00001     # Time step (reduced for CFL stability)
N_steps = int(T_final / dt)

# Domain decomposition
N_local = N_global // size
dx = L / (N_global - 1)

# CFL condition check
cfl = alpha * dt / dx**2
if rank == 0:
    print(f"CFL number: {cfl:.4f} (should be < 0.5 for stability)")
    if cfl >= 0.5:
        print("WARNING: Unstable parameters!")

# Local grid (with ghost cells for boundaries)
u = np.zeros(N_local + 2)
u_new = np.zeros(N_local + 2)

# Initialize temperature distribution
x_local = np.linspace(rank * N_local * dx, (rank + 1) * N_local * dx, N_local + 2)
u[:] = np.sin(np.pi * x_local / L)  # Initial condition: sin wave

# Determine neighbors
left_neighbor = rank - 1 if rank > 0 else MPI.PROC_NULL
right_neighbor = rank + 1 if rank < size - 1 else MPI.PROC_NULL

# Time evolution
for step in range(N_steps):
    # Exchange boundary values with neighbors
    # Send right boundary to right neighbor, receive left boundary from left neighbor
    comm.Sendrecv(u[-2:-1], dest=right_neighbor, sendtag=0,
                  recvbuf=u[0:1], source=left_neighbor, recvtag=0)
    
    # Send left boundary to left neighbor, receive right boundary from right neighbor
    comm.Sendrecv(u[1:2], dest=left_neighbor, sendtag=1,
                  recvbuf=u[-1:], source=right_neighbor, recvtag=1)
    
    # Update interior points using finite difference
    u_new[1:-1] = u[1:-1] + cfl * (u[2:] - 2*u[1:-1] + u[:-2])
    
    # Boundary conditions (fixed at 0)
    if rank == 0:
        u_new[1] = 0.0
    if rank == size - 1:
        u_new[-2] = 0.0
    
    # Swap arrays
    u, u_new = u_new, u

# Gather results for plotting
u_global = None
if rank == 0:
    u_global = np.zeros(N_global)

# Remove ghost cells before gathering
u_local = u[1:-1]
comm.Gather(u_local, u_global, root=0)

if rank == 0:
    # Plot results
    x = np.linspace(0, L, N_global)
    plt.figure(figsize=(10, 6))
    plt.plot(x, np.sin(np.pi * x / L), 'b--', label='Initial', linewidth=2)
    plt.plot(x, u_global, 'r-', label=f't = {T_final}', linewidth=2)
    plt.xlabel('x', fontsize=12)
    plt.ylabel('Temperature', fontsize=12)
    plt.title(f'1D Heat Equation (Parallel with {size} processes)', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('heat_equation_parallel.png', dpi=150)
    print(f"\\nSimulation complete! Plot saved as 'heat_equation_parallel.png'")
    print(f"Steps computed: {N_steps:,}")
    print(f"Grid points per process: {N_local}")
"""

with open('mpi_heat_equation.py', 'w') as f:
    f.write(heat_eq_code)

print("Created: mpi_heat_equation.py")
print("\nRun with: mpiexec -n 4 python mpi_heat_equation.py")
print("\nNote: N_global=1004 (divisible by 4), dt reduced for CFL stability")

Created: mpi_heat_equation.py

Run with: mpiexec -n 4 python mpi_heat_equation.py


---

## 9. Performance Analysis

### Key Metrics

1. **Speedup**: S(p) = T₁ / Tₚ
   - T₁: Time with 1 process
   - Tₚ: Time with p processes
   - Ideal: S(p) = p (linear speedup)

2. **Efficiency**: E(p) = S(p) / p = T₁ / (p × Tₚ)
   - Ideal: E(p) = 1 (100%)
   - Typical: E(p) = 0.7-0.9 (70-90%)

3. **Scalability**: How speedup changes with more processes
   - **Strong scaling**: Fixed problem size, vary processes
   - **Weak scaling**: Fixed problem size per process

### Amdahl's Law

Not all code can be parallelized. If fraction `f` is serial:

**Speedup ≤ 1 / (f + (1-f)/p)**

Example: If 10% is serial (f=0.1):
- With 10 processes: Max speedup ≈ 5.3×
- With 100 processes: Max speedup ≈ 9.2×
- With ∞ processes: Max speedup = 10×

**Implication**: Even small serial portions limit scalability!

In [12]:
# Create a benchmark script to measure scaling
benchmark_code = """from mpi4py import MPI
import numpy as np
import time

def benchmark_computation(size):
    \"\"\"Perform some computation to benchmark\"\"\"
    N = size
    A = np.random.randn(N, N)
    B = np.random.randn(N, N)
    C = A @ B
    return np.sum(C)

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Problem sizes to test
problem_sizes = [500, 1000, 2000, 4000]

if rank == 0:
    print(f"{'='*70}")
    print(f"MPI Scaling Benchmark")
    print(f"Number of processes: {size}")
    print(f"{'='*70}")

for N in problem_sizes:
    comm.Barrier()
    start_time = time.time()
    
    # Each process does some work
    local_result = benchmark_computation(N // size)
    
    # Combine results
    global_result = comm.reduce(local_result, op=MPI.SUM, root=0)
    
    comm.Barrier()
    elapsed = time.time() - start_time
    
    if rank == 0:
        print(f"\\nProblem size: {N}×{N}")
        print(f"  Time: {elapsed:.4f} seconds")
        print(f"  Time per process: {elapsed:.4f} s")
        
print(f"\\nRank {rank} finished benchmark")
"""

with open('mpi_benchmark.py', 'w') as f:
    f.write(benchmark_code)

print("Created: mpi_benchmark.py")
print("\nTo measure scaling, run with different numbers of processes:")
print("  mpiexec -n 1 python mpi_benchmark.py")
print("  mpiexec -n 2 python mpi_benchmark.py")
print("  mpiexec -n 4 python mpi_benchmark.py")
print("  mpiexec -n 8 python mpi_benchmark.py")

Created: mpi_benchmark.py

To measure scaling, run with different numbers of processes:
  mpiexec -n 1 python mpi_benchmark.py
  mpiexec -n 2 python mpi_benchmark.py
  mpiexec -n 4 python mpi_benchmark.py
  mpiexec -n 8 python mpi_benchmark.py


---

## 10. Best Practices and Common Patterns

### Design Patterns for MPI

1. **Manager-Worker Pattern**
   - Rank 0 distributes work and collects results
   - Other ranks process tasks
   - Good for embarrassingly parallel problems

2. **SPMD (Single Program Multiple Data)**
   - All processes run the same code
   - Behavior depends on rank
   - Most common MPI pattern

3. **Pipeline Pattern**
   - Data flows through processes in sequence
   - Each process performs a stage of computation
   - Good for streaming data processing

### Debugging MPI Programs

**Common issues:**
1. ⚠️ **Deadlocks**: Use `comm.Barrier()` to synchronize and debug
2. ⚠️ **Wrong tags/ranks**: Print rank and size to verify
3. ⚠️ **Buffer size mismatches**: Check array sizes match
4. ⚠️ **Race conditions**: Use proper synchronization

**Debugging tools:**
- Print statements with rank information
- Use `comm.Barrier()` to isolate problems
- Run with small number of processes first (2-4)
- Use `MPI.COMM_WORLD.Get_attr(MPI.TAG_UB)` to check max tag value

### Code Organization Tips

```python
# Good practice: Separate MPI logic from computation
def compute_physics(data):
    \"\"\"Pure computation - no MPI\"\"\"
    return result

def parallel_workflow():
    \"\"\"MPI communication logic\"\"\"
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    
    # Distribute data
    local_data = distribute_data(comm, rank)
    
    # Compute (MPI-free)
    local_result = compute_physics(local_data)
    
    # Gather results
    return gather_results(comm, rank, local_result)
```

### When to Use MPI

✅ **Good for:**
- Large-scale simulations on clusters
- Problems requiring distributed memory
- Long-running computations
- Production scientific codes

❌ **Avoid when:**
- Problem fits in single-node memory
- Development/prototyping phase
- Interactive analysis
- Overhead > computation time

**Alternatives:**
- **Small problems**: NumPy/SciPy (optimized serial code)
- **Shared memory**: OpenMP, threading
- **GPU acceleration**: CuPy, PyTorch, JAX
- **Task parallelism**: Dask, Ray

---

## 11. Summary

### What We Learned

1. ✅ **MPI Basics**
   - Communicators, ranks, and process organization
   - Point-to-point communication (send/recv)
   - Collective operations (bcast, scatter, gather, reduce)

2. ✅ **Practical Skills**
   - Writing and running MPI programs with mpi4py
   - Parallel numerical algorithms (integration, matrix operations)
   - Solving PDEs with domain decomposition

3. ✅ **Advanced Concepts**
   - Non-blocking communication
   - Performance analysis and scaling
   - Common pitfalls and debugging

4. ✅ **Physics Applications**
   - Monte Carlo simulations
   - Heat equation solver
   - Matrix-vector multiplication

### Key Takeaways

💡 **MPI is powerful but requires careful design**
- Think about data distribution and communication patterns
- Minimize communication overhead
- Balance computational load

💡 **Start simple, then optimize**
- Get it working correctly first
- Measure performance before optimizing
- Profile to find bottlenecks

💡 **Not every problem needs MPI**
- Consider the problem size and available resources
- Sometimes serial or shared-memory parallelism is better
- Choose the right tool for the job

---

## 12. Additional Resources

### Documentation
- **mpi4py**: https://mpi4py.readthedocs.io/
- **MPI Standard**: https://www.mpi-forum.org/
- **MPI Tutorial**: https://mpitutorial.com/

### Books
- *"Parallel Programming with MPI"* by Peter Pacheco
- *"Using MPI"* by Gropp, Lusk, and Skjellum
- *"Python for High Performance Computing"* by various authors

### Online Courses
- XSEDE/TACC MPI tutorials
- LLNL HPC tutorials
- Coursera: Parallel Programming courses

### Tools
- **Performance**: `mprof`, Intel VTune, Scalasca
- **Debugging**: `gdb` with MPI support, Allinea DDT
- **Profiling**: `mpiP`, TAU

---

## 13. Exercises and Practice Problems

### Exercise 1: Parallel Dot Product
Write an MPI program to compute the dot product of two large vectors in parallel.
- Distribute vectors using `scatter`
- Each process computes local dot product
- Use `reduce` with `MPI.SUM` to get final result

### Exercise 2: Parallel Sorting
Implement parallel merge sort:
- Each process sorts its local data
- Use point-to-point communication to merge results
- Compare performance with serial sort

### Exercise 3: 2D Heat Equation
Extend the 1D heat equation to 2D:
- Use 2D domain decomposition
- Exchange boundary data with 4 neighbors
- Visualize the temperature field

### Exercise 4: Parallel Random Walk
Simulate random walks in parallel:
- Each process simulates independent walkers
- Compute statistics (mean displacement, variance)
- Use `gather` to collect all trajectories

### Exercise 5: Mandelbrot Set
Compute the Mandelbrot set in parallel:
- Divide the complex plane among processes
- Each process computes its portion
- Gather results and create visualization

---

## 14. Running the Examples

All example files created in this notebook can be executed from the terminal:

```bash
# Basic examples
mpiexec -n 4 python mpi_hello_example.py
mpiexec -n 2 python mpi_send_recv.py
mpiexec -n 4 python mpi_collective.py

# Numerical applications
mpiexec -n 4 python mpi_pi_monte_carlo.py
mpiexec -n 4 python mpi_trapezoid.py
mpiexec -n 4 python mpi_matvec.py

# Advanced examples
mpiexec -n 2 python mpi_nonblocking.py
mpiexec -n 4 python mpi_heat_equation.py
mpiexec -n 4 python mpi_benchmark.py
```

**Tips:**
- Start with small number of processes (2-4) for testing
- Increase to match your CPU cores for real runs
- On clusters, use job submission scripts (SLURM, PBS)
- Monitor resource usage with `top` or `htop`

---

## 15. Quick Reference: mpi4py Cheat Sheet

### Initialization
```python
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
```

### Point-to-Point Communication
| Method | Description | Use Case |
|--------|-------------|----------|
| `comm.send(obj, dest, tag)` | Send Python object | General data |
| `comm.recv(source, tag)` | Receive Python object | General data |
| `comm.Send(buf, dest, tag)` | Send NumPy array | Fast, large arrays |
| `comm.Recv(buf, source, tag)` | Receive NumPy array | Fast, large arrays |
| `comm.isend(obj, dest, tag)` | Non-blocking send | Overlap comm/comp |
| `comm.irecv(source, tag)` | Non-blocking receive | Overlap comm/comp |

### Collective Communication
| Method | Description | Pattern |
|--------|-------------|---------|
| `comm.bcast(obj, root)` | Broadcast from root | 1 → all |
| `comm.scatter(data, root)` | Distribute data | 1 → all (split) |
| `comm.gather(data, root)` | Collect data | all → 1 |
| `comm.allgather(data)` | Gather to all | all → all |
| `comm.reduce(data, op, root)` | Reduce to root | all → 1 (combine) |
| `comm.allreduce(data, op)` | Reduce to all | all → all (combine) |

### Reduction Operations
```python
MPI.SUM     # Sum values
MPI.MAX     # Maximum value
MPI.MIN     # Minimum value
MPI.PROD    # Product
MPI.LAND    # Logical AND
MPI.LOR     # Logical OR
```

### Synchronization
```python
comm.Barrier()  # Wait for all processes
request.wait()  # Wait for non-blocking operation
```

### Common Patterns
```python
# Master-worker
if rank == 0:
    # Master process
    pass
else:
    # Worker processes
    pass

# Domain decomposition
local_n = n // size
local_start = rank * local_n
local_end = (rank + 1) * local_n
```

---

## 🎓 End of Lecture

**Thank you!**

Questions? Experiments? Bugs? 

Let's discuss and explore MPI together!

---

**Next Steps:**
1. Run the example programs
2. Try the exercises
3. Apply MPI to your research problems
4. Explore advanced features (custom communicators, derived datatypes)
5. Profile and optimize your codes

**Remember**: The best way to learn MPI is to use it on real problems!

In [13]:
# List all MPI example files created
import os
import glob

print("=" * 70)
print("MPI Example Files Created")
print("=" * 70)

# Get all .py files in current directory
py_files = sorted([f for f in glob.glob("mpi_*.py") if os.path.isfile(f)])

if py_files:
    print(f"\nFound {len(py_files)} MPI example files:\n")
    for i, filename in enumerate(py_files, 1):
        size = os.path.getsize(filename)
        print(f"  {i}. {filename:<30} ({size:,} bytes)")
    
    print("\n" + "=" * 70)
    print("To run any example:")
    print("  mpiexec -n <num_processes> python <filename>")
    print("\nExample:")
    print("  mpiexec -n 4 python mpi_hello_example.py")
    print("=" * 70)
else:
    print("\nNo MPI files found. Run the code cells above to create them.")
    print("=" * 70)

MPI Example Files Created

Found 9 MPI example files:

  1. mpi_benchmark.py               (1,057 bytes)
  2. mpi_collective.py              (1,580 bytes)
  3. mpi_heat_equation.py           (2,840 bytes)
  4. mpi_hello_example.py           (464 bytes)
  5. mpi_matvec.py                  (2,065 bytes)
  6. mpi_nonblocking.py             (1,408 bytes)
  7. mpi_pi_monte_carlo.py          (1,188 bytes)
  8. mpi_send_recv.py               (892 bytes)
  9. mpi_trapezoid.py               (1,599 bytes)

To run any example:
  mpiexec -n <num_processes> python <filename>

Example:
  mpiexec -n 4 python mpi_hello_example.py


---

## 16. Bug Fixes and Testing Results

### Issues Found and Fixed

During testing with `mpirun -np 4 python`, two issues were discovered and fixed:

#### 1. **mpi_matvec.py** - Communication Method Issues

**Problem:**
- Original code used uppercase MPI methods (`Scatter`, `Bcast`, `Gather`) which require explicit buffer specifications
- The buffer arguments were not correctly specified, causing the program to hang indefinitely
- The program worked conceptually but had implementation issues with memory buffers

**Solution:**
- Converted to lowercase methods (`scatter`, `bcast`, `gather`) which handle data automatically via Python's pickle protocol
- These methods are more Pythonic and handle object serialization transparently
- Reduced matrix size from 10,000 to 1,000 for faster execution during testing

**Code Changes:**
```python
# BEFORE (problematic):
comm.Scatter(A, local_A, root=0)
comm.Bcast(x_local if rank != 0 else x, root=0)
comm.Gather(local_y, y_parallel, root=0)

# AFTER (fixed):
if rank == 0:
    A_chunks = [A[i*rows_per_process:(i+1)*rows_per_process] for i in range(size)]
else:
    A_chunks = None
local_A = comm.scatter(A_chunks, root=0)
x_local = comm.bcast(x if rank == 0 else None, root=0)
y_parallel = comm.gather(local_y, root=0)
if rank == 0:
    y_parallel = np.concatenate(y_parallel)
```

**Result:** ✓ Program runs successfully with 0.00e+00 relative error

---

#### 2. **mpi_heat_equation.py** - Multiple Numerical Issues

**Problem 1: Array Size Mismatch**
- `N_global = 1000` is not evenly divisible by 4 processes
- The `Gather` operation requires exact buffer sizes
- Error: `ValueError: number of entries 1002 is not a multiple of required number of blocks 4`

**Solution:** Changed `N_global = 1004` (divisible by 4)

**Problem 2: CFL Stability**
- CFL number = α × dt / dx² = 0.9980 (should be < 0.5 for stability)
- This caused numerical instability (overflow, NaN values)
- The simulation would blow up instead of converging

**Solution:** Reduced `dt` from 0.0001 to 0.00001, bringing CFL to 0.1006

**Problem 3: Gather Buffer Size**
- Gather buffer was allocated as `N_global + 2` but should be `N_global`
- Ghost cells were already removed before gathering

**Solution:** Changed buffer allocation from `np.zeros(N_global + 2)` to `np.zeros(N_global)`

**Result:** ✓ Simulation completes successfully, plot saved

---

### Testing Summary

All MPI example programs were tested with `mpirun -np 4 python <script>.py`:

| Program | Status | Notes |
|---------|--------|-------|
| `mpi_hello_example.py` | ✅ Pass | Works correctly |
| `mpi_send_recv.py` | ✅ Pass | Point-to-point communication working |
| `mpi_collective.py` | ✅ Pass | All collective operations working |
| `mpi_nonblocking.py` | ✅ Pass | Non-blocking communication working |
| `mpi_trapezoid.py` | ✅ Pass | π ≈ 3.1415926536 (exact!) |
| `mpi_pi_monte_carlo.py` | ✅ Pass | π ≈ 3.1416360800 (error: 4.3×10⁻⁵) |
| `mpi_matvec.py` | ✅ **Fixed** | Switched to lowercase scatter/bcast/gather |
| `mpi_benchmark.py` | ✅ Pass | Scaling benchmark working |
| `mpi_heat_equation.py` | ✅ **Fixed** | Fixed N_global, dt, and buffer size |

### Key Lessons Learned

1. **Use lowercase methods for flexibility**: `scatter`, `bcast`, `gather` are easier to use than their uppercase counterparts for general Python objects and arrays

2. **Check divisibility**: When distributing data among processes, ensure the problem size is evenly divisible by the number of processes, or handle remainders explicitly

3. **Verify CFL condition**: For explicit time-stepping methods in PDEs, always check that the CFL condition is satisfied for numerical stability

4. **Buffer size management**: Be careful with buffer sizes in collective operations - they must match exactly between send and receive sides

5. **Test with small process counts first**: Running with 2-4 processes makes debugging much easier than starting with large-scale runs

---

### Performance Results

**mpi_matvec.py** (1000×1000 matrix, 4 processes):
- Serial time: 0.0010 seconds
- Parallel time: 0.0076 seconds
- Speedup: 0.13× (slower due to communication overhead for small problem)
- Relative error: 0.00e+00 ✓

**mpi_pi_monte_carlo.py** (100M samples, 4 processes):
- Time: 0.37 seconds
- Throughput: 270M samples/sec
- π error: 4.3×10⁻⁵

**mpi_trapezoid.py** (10M trapezoids, 4 processes):
- Time: 0.0084 seconds
- π error: 0.00e+00 ✓

**mpi_heat_equation.py** (1004 grid points, 4 processes):
- Steps: 49,999
- Grid points per process: 251
- Simulation completed successfully ✓

---

## 17. When to Use Uppercase vs Lowercase MPI Methods

### Understanding the Two Flavors

mpi4py provides two sets of communication methods:

#### **Lowercase Methods** (`send`, `recv`, `scatter`, `bcast`, etc.)

**Characteristics:**
- Use Python's pickle protocol for serialization
- Work with any Python object (lists, dicts, custom classes)
- Automatic type handling and buffer management
- More Pythonic and easier to use
- Slower due to serialization overhead

**Best for:**
- ✅ Small to medium-sized data transfers
- ✅ Complex Python objects (dictionaries, lists of mixed types)
- ✅ Prototyping and development
- ✅ When convenience matters more than performance
- ✅ Irregular data structures

**Example:**
```python
# Easy and flexible
if rank == 0:
    data = {'config': [1, 2, 3], 'params': {'alpha': 0.1}}
else:
    data = None
data = comm.bcast(data, root=0)
```

---

#### **Uppercase Methods** (`Send`, `Recv`, `Scatter`, `Bcast`, etc.)

**Characteristics:**
- Work directly with memory buffers (NumPy arrays)
- No serialization - direct memory copies
- Requires explicit buffer specification
- Much faster for large numerical arrays
- More complex to use correctly

**Best for:**
- ✅ Large NumPy arrays (> 1 MB)
- ✅ Production code where performance is critical
- ✅ Repeated communication in tight loops
- ✅ When you need maximum speed
- ✅ HPC applications with massive data

**Example:**
```python
# Fast but requires careful buffer management
if rank == 0:
    data = np.random.randn(1000000)
else:
    data = np.empty(1000000, dtype=np.float64)
comm.Bcast([data, MPI.DOUBLE], root=0)  # Explicit buffer spec
```

---

### Performance Comparison

| Data Size | Lowercase (pickle) | Uppercase (buffer) | Speedup |
|-----------|-------------------|-------------------|---------|
| 1 KB | ~0.1 ms | ~0.05 ms | 2× |
| 1 MB | ~10 ms | ~1 ms | 10× |
| 100 MB | ~1000 ms | ~50 ms | 20× |
| 1 GB | ~10 sec | ~0.5 sec | 20× |

**Rule of thumb:** For arrays larger than 1 MB, uppercase methods are significantly faster.

---

### Recommendations

1. **Start with lowercase methods** during development
   - Get the algorithm working first
   - Easier to debug and understand
   - Less prone to buffer size errors

2. **Switch to uppercase for optimization**
   - Profile your code first
   - Only optimize communication bottlenecks
   - Use for large arrays in production code

3. **Hybrid approach**
   - Use lowercase for control messages and metadata
   - Use uppercase for bulk data transfers
   - Example:
   ```python
   # Send small config with lowercase
   config = comm.bcast(config_dict, root=0)
   
   # Send large array with uppercase
   comm.Bcast([large_array, MPI.DOUBLE], root=0)
   ```

4. **Common pitfalls to avoid**
   - ❌ Don't use uppercase methods with Python lists
   - ❌ Don't forget to pre-allocate receive buffers for uppercase
   - ❌ Don't mix uppercase and lowercase in same communication
   - ❌ Don't use uppercase for small, infrequent messages

---

### Updated Best Practices

Based on our bug fixes and testing:

✅ **DO:**
- Use lowercase methods for general-purpose code
- Ensure problem sizes are divisible by process count
- Check numerical stability (CFL conditions, etc.)
- Test with small process counts first (2-4)
- Verify buffer sizes match for collective operations
- Use `comm.Barrier()` for synchronization when debugging

❌ **DON'T:**
- Don't assume uppercase methods are always faster (overhead matters)
- Don't use incorrect buffer specifications
- Don't ignore numerical stability requirements
- Don't parallelize everything (Amdahl's law!)
- Don't forget to handle remainders in data distribution

---

### Example: Correct Usage of Both Methods

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Lowercase for small metadata
if rank == 0:
    metadata = {'problem_size': 10000, 'timesteps': 1000, 'dt': 0.01}
else:
    metadata = None
metadata = comm.bcast(metadata, root=0)

# Uppercase for large arrays
N = metadata['problem_size']
if rank == 0:
    large_array = np.random.randn(N, N)
else:
    large_array = np.empty((N, N), dtype=np.float64)

# Note: For scatter/gather with uppercase, it's complex
# Easier to use lowercase for scatter/gather:
if rank == 0:
    chunks = [large_array[i::size] for i in range(size)]
else:
    chunks = None
local_data = comm.scatter(chunks, root=0)

# Process local data...
result = np.sum(local_data)

# Gather results (lowercase is easier)
all_results = comm.gather(result, root=0)
```

---

## ✅ Testing Complete - All Programs Working!

### Summary of Changes

All MPI example programs have been tested with `mpirun -np 4 python` and are now **fully functional**. 

**Two bugs were fixed:**

1. **`mpi_matvec.py`**: Converted from uppercase to lowercase MPI methods for automatic buffer management
2. **`mpi_heat_equation.py`**: Fixed grid size divisibility, CFL stability, and buffer allocation

### Verification Commands

You can verify all programs work correctly by running these commands in your terminal:

```bash
# Navigate to lecture directory
cd lectures/20251121_lecture7

# Test all programs (should all run successfully)
mpirun -np 4 python mpi_hello_example.py
mpirun -np 2 python mpi_send_recv.py
mpirun -np 4 python mpi_collective.py
mpirun -np 2 python mpi_nonblocking.py
mpirun -np 4 python mpi_trapezoid.py
mpirun -np 4 python mpi_pi_monte_carlo.py
mpirun -np 4 python mpi_matvec.py
mpirun -np 4 python mpi_benchmark.py
mpirun -np 4 python mpi_heat_equation.py
```

### What You Should See

✅ **All programs complete without errors**  
✅ **Numerical results are accurate** (π estimates, matrix operations)  
✅ **No deadlocks or hanging**  
✅ **Proper parallel speedup** (for appropriate problem sizes)  

---

**Happy Parallel Computing! 🚀**

Remember: Start simple, test thoroughly, then scale up!